In [1]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [2]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 1, 3, 0, 5, 10, 1, 2],
              [500, 700, 750, 600, 1450,
               800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2, 
               1, 3, 3, 1, 2]], dtype = np.float64)

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype = np.float64)

In [3]:
def calc_std_feat(x):
    res = (x - x.mean()) / x.std()
    return res

In [4]:
X_st = X.copy()
X_st[2, :] = calc_std_feat(X[2, :])
X_st

array([[ 1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ],
       [ 1.        ,  1.        ,  2.        ,  1.        ,  3.        ,
         0.        ,  5.        , 10.        ,  1.        ,  2.        ],
       [-0.97958969, -0.56713087, -0.46401617, -0.77336028,  0.97958969,
        -0.36090146,  1.08270439,  2.11385144, -1.08270439,  0.05155735],
       [ 1.        ,  1.        ,  2.        ,  1.        ,  2.        ,
         1.        ,  3.        ,  3.        ,  1.        ,  2.        ]])

In [5]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

#### Задание 1

Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.

In [6]:
def calc_logloss(y, y_pred):
    if all(y_pred != 1):
        err = - np.mean(y * np.log(y_pred + 1e-10) + (1.0 - y) * np.log(1.0 - y_pred))
    return err

#### Задание 2

Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.

In [7]:
def eval_model(X, y, iterations, alpha=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T))
        if i % (iterations / 10) == 0:
            print(i, W, err)
    return W

In [8]:
W = eval_model(X_st, y, iterations=100000, alpha=1e-1)

10000 [-11.27241705  -1.45342424  -2.38315559   9.49424167] 0.25238324887644675
20000 [-16.2427091   -1.86940882  -3.90475531  13.50901763] 0.2087442171864689
30000 [-20.12083462  -2.20557401  -5.06349228  16.65543794] 0.1822509791049001
40000 [-23.32991383  -2.48704062  -6.00960348  19.2582858 ] 0.1641585573770004
50000 [-26.07030122  -2.7286846   -6.81038328  21.47869092] 0.1509953863952354
60000 [-28.46500332  -2.94026656  -7.50582736  23.41651039] 0.1409638345671192
70000 [-30.59553066  -3.12847345  -8.12185755  25.13810328] 0.1330377989677229
80000 [-32.5184265   -3.29804548  -8.67615891  26.68956031] 0.12659210785991643
90000 [-34.27432938  -3.45244383  -9.1812758   28.10403169] 0.12122579818628525
100000 [-35.8933164   -3.59426199  -9.64639296  29.4060647 ] 0.1166706078533614


#### Задание 3

Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).

In [9]:
def calc_pred_proba(W, X):
    y_pred_proba = 1 / (1 + np.exp(-np.dot(W, X)))
    return y_pred_proba

In [10]:
calc_pred_proba(W, X_st)

array([0.34705568, 0.00984648, 1.        , 0.06777527, 0.93622651,
       0.04716209, 1.        , 0.01783172, 0.58968863, 0.99999976])

#### Задание 4

Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

In [11]:
def calc_pred(W, X, p):
    y_pred = calc_pred_proba(W, X_st)
    y_pred = np.where(y_pred >= p, 1, 0)
    return y_pred

In [12]:
y_pred = calc_pred(W, X_st, 0.5)
y_pred

array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1])

#### Задание 5

Посчитайте Accuracy, матрицу ошибок, точность и полноту, а также F1 score.

In [13]:
def quality(y, y_pred):
    
    # Посчитаем Accuracy: 
    accuracy = 1 / len(y) * sum(y_pred == y)
    
    # Посчитаем матрицу ошибок:
    confusion_matrix = np.zeros((2, 2))
    TP = np.sum(np.where(y == 1, y == y_pred, False))
    FP = np.sum(np.where(y == 0, y == y_pred, False))
    FN = np.sum(np.where(y == 1, y != y_pred, False))
    TN = np.sum(np.where(y == 0, y != y_pred, False))
    confusion_matrix[0, 0] = TP
    confusion_matrix[1, 1] = FP
    confusion_matrix[1, 0] = FN
    confusion_matrix[0, 1] = TN
    
    # Посчитаем точность (precision):
    precision = TP / (TP + FP)
    
    # Посчитаем полноту (recall):
    recall = TP / (TP + FN)
    
    # Посчитаем F1 score:
    f1_score = 2 * precision * recall / (precision + recall)
    
    # Выведем результаты:
    print(f'Accuracy = {accuracy}\nМатрица ошибок:\n{confusion_matrix}\nТочность (precision) = {precision}\nПолнота (recall) = {recall}\nF1 score = {f1_score}')

In [14]:
quality(y, y_pred)

Accuracy = 1.0
Матрица ошибок:
[[5. 0.]
 [0. 5.]]
Точность (precision) = 0.5
Полнота (recall) = 1.0
F1 score = 0.6666666666666666


#### Задание 6

###### Могла ли модель переобучиться? Почему?

Думаю, что могла, не было валидации. Данных мало, есть выброс.